In [19]:
import numpy as np; import pandas as pd
import xarray as xr; import matplotlib.pyplot as plt;
from assemble_class import assemble
import sqlite3

In [127]:
conn = sqlite3.connect('LoaderDB3.db') # connection
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [73]:
def Prepare_id(**kwargs):
    Data = "SELECT * FROM OBJ_ID"
    i = 0
    for key, value in kwargs.items():
        if i == 0:
            Data += " WHERE "
        else:
            Data += " AND "
        Data += "{}='{}'".format(key, value)
        i += 1
    Data += ";"
    return c.execute(Data)


def format_change(DICT):
    limits_keys = ['t0','t1','lon','lat']
    for n in range(len(DICT)):
        DICT[n]['limits'] = {key: DICT[n][key] for key in DICT[n] if key in limits_keys}
        for key in limits_keys: del DICT[n][key]

In [74]:
def calis():
    c.execute

In [128]:
Prepare_id(t0= '2018-09-15', name = 'SIO1')

In [129]:
sio1_id = [dict(row) for row in c.fetchall()] 

In [130]:
sio1_id

[{'name': 'SIO1',
  'type': 'MOORING',
  'project': 'PISTON',
  't0': '2018-09-15',
  't1': '2019-10-31',
  'lon': None,
  'lat': None,
  'COMBINER_ID': 102}]

In [131]:
format_change(sio1_id)

In [132]:
sio1_id[0]

{'name': 'SIO1',
 'type': 'MOORING',
 'project': 'PISTON',
 'COMBINER_ID': 102,
 'limits': {'t0': '2018-09-15', 't1': '2019-10-31', 'lon': None, 'lat': None}}

In [133]:
combiner_id = sio1_id[0]['COMBINER_ID']

In [134]:
print("hola %d" %combiner_id)

hola 102


In [141]:
c.execute('SELECT * FROM COMBINER AS c INNER JOIN DATASOURCES as d ON c.Data_obj = d.Data_obj WHERE c.COMBINER_ID = %d' %combiner_id)
result  = [dict(row) for row in c.fetchall()] 

In [142]:
result

[{'COMBINER_ID': 102,
  'DATA_OBJ': 201,
  'Source': 'chipod',
  'Filepath': 'sio1_chipod.mat'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 202,
  'Source': 'chiflag',
  'Filepath': 'SIO1_chipod_finflag.csv'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 301,
  'Source': 'ADCP',
  'Filepath': 'SIO1_4021.nc'},
 {'COMBINER_ID': 102,
  'DATA_OBJ': 302,
  'Source': 'ADCP',
  'Filepath': 'SIO1_11181.nc'}]

In [147]:
c.execute('SELECT * FROM COMBINER AS c INNER JOIN DATASOURCES as d ON c.Data_obj = d.Data_obj WHERE c.COMBINER_ID = %d AND d.Source = "chipod"' %combiner_id)
result2  = [dict(row) for row in c.fetchall()] 

In [148]:
result2

[{'COMBINER_ID': 102,
  'DATA_OBJ': 201,
  'Source': 'chipod',
  'Filepath': 'sio1_chipod.mat'}]

In [152]:
result[0]

{'COMBINER_ID': 102,
 'DATA_OBJ': 201,
 'Source': 'chipod',
 'Filepath': 'sio1_chipod.mat'}

In [158]:
for kk in range(len(result)):
    print(result[kk])

{'COMBINER_ID': 102, 'DATA_OBJ': 201, 'Source': 'chipod', 'Filepath': 'sio1_chipod.mat'}
{'COMBINER_ID': 102, 'DATA_OBJ': 202, 'Source': 'chiflag', 'Filepath': 'SIO1_chipod_finflag.csv'}
{'COMBINER_ID': 102, 'DATA_OBJ': 301, 'Source': 'ADCP', 'Filepath': 'SIO1_4021.nc'}
{'COMBINER_ID': 102, 'DATA_OBJ': 302, 'Source': 'ADCP', 'Filepath': 'SIO1_11181.nc'}


In [171]:
sio1_paths = {}

In [163]:
sio1_paths[result[0]['Source']] = result[0]['Filepath']

In [179]:
sio1_paths

{'chipod': '/mnt/sda1/SciData/PISTON/Moorings/SIO1/sio1_chipod.mat',
 'chiflag': '/mnt/sda1/SciData/PISTON/Moorings/SIO1/SIO1_chipod_finflag.csv',
 'ADCP': ['/mnt/sda1/SciData/PISTON/Moorings/SIO1/ADC/SIO1_11181.nc']}

In [165]:
sio1_data_dir = '/mnt/sda1/SciData/PISTON/Moorings/SIO1/'

In [166]:
sio1_paths[result[0]['Source']] = sio1_data_dir + result[0]['Filepath']

In [178]:
for kk in range(len(result)):
    if result[kk]['Source'] != 'ADCP':
        sio1_paths[result[kk]['Source']] = sio1_data_dir + result[kk]['Filepath']
    else:
        sio1_paths[result[kk]['Source']] = [sio1_data_dir + 'ADC/' + result[kk]['Filepath']]